In [ ]:
from module import TransformerDecoder
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch

In [ ]:
from char_dataset import CharDataset 
shakespeare_dataset = CharDataset("shakespeare.txt", block_size=128)

In [ ]:
from torch.utils.data import DataLoader
dataloader = DataLoader(shakespeare_dataset, batch_size=32, shuffle=True)

# Training

In [ ]:
decoder = TransformerDecoder(vocab_size=shakespeare_dataset.get_vocab_size(),
                           d_model=128,
                           n_layers=12,
                           n_heads=8,
                           d_ff=100,
                           max_len=512,
                           dropout=0.1)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(decoder.parameters(), lr=0.001)

for epoch in range(1):
    for input_data, target_data in dataloader:
        optimizer.zero_grad() 
        output = decoder(input_data)
		loss = criterion(output, target_data)
		loss.backward() 
		optimizer.step()